In [ ]:
import argparse
from argparse import RawTextHelpFormatter
import numpy as np
import os

# keras
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Reshape, UpSampling2D, Conv2DTranspose, Flatten, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.optimizers import Adam


from absl import app

# logging
import wandb
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib import gridspec, colors
import shutil
from glob import glob
from sklearn.metrics import mean_squared_error as mse

## Load models

In [ ]:
encoder = load_model("/home/aogorodnikov/aae/encoder.h5")
decoder = load_model("/home/aogorodnikov/aae/decoder.h5")
discriminator = load_model("/home/aogorodnikov/aae/discriminator.h5")
generator = load_model("/home/aogorodnikov/aae/generator.h5")

In [ ]:
models = "/home/aogorodnikov/aae_5k"
wd = "/home/aogorodnikov/test"
batch = 16

In [ ]:
def model_compile(models):
    '''
    Loads and compiles models
    :param models: directory with encoder, decoder and discriminator

    :return: compiled autoencoder and discriminator
    '''

    encoder = load_model(os.path.join(models, "encoder.h5"))
    decoder = load_model(os.path.join(models, "decoder.h5"))
    discriminator = load_model(os.path.join(models, "discriminator.h5"))

    # get input and latent space shapes
    latent_dim = encoder.get_layer(index=-1).output_shape[1]
    input_dim = encoder.get_layer(index=0).input_shape[1:]

    # compile imported models into assembled autoencoder
    autoencoder_input = Input(shape=input_dim)
    autoencoder = Model(autoencoder_input, decoder(encoder(autoencoder_input)))
    autoencoder.compile(optimizer=Adam(lr=1e-4), loss="mean_squared_error", metrics=['accuracy'])
    discriminator.compile(optimizer=Adam(lr=1e-4), loss="binary_crossentropy", metrics=['accuracy'])

    return autoencoder, discriminator, input_dim, latent_dim

In [5]:
autoencoder, discriminator, input_dim, latent_dim = model_compile(models)

In [6]:
data_loader = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    shear_range=0,
    zoom_range=0,
    horizontal_flip=False)
# load data
data_in = data_loader.flow_from_directory(
    wd,
    target_size=(input_dim[0], input_dim[0]),
    batch_size=batch,
    shuffle=False,
    class_mode='input')

Found 358 images belonging to 1 classes.


In [7]:
data_in.filepaths[:17]

['/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f01_0231.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f01_0433.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f01_0511.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f02_0616.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f02_0711.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f03_0368.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f03_0619.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f03_0650.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f04_0574.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f04_0586.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f06_0424.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f07_0124.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f07_0421.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f08_0400.tif',
 '/home/aogorodnikov/test/micronuclei/Pt04_r01c01_f11_0103.tif',
 '/home/aogorodnikov/test

In [8]:
batch_index = 0

data = data_in.next()
data_list = data[0]
data_size = len(data_list)

ae_pred = autoencoder.predict_on_batch(data_list)

/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [9]:
def mse_batch(data_x, data_y, input_dim):
    shape_1 = input_dim[0]
    shape_2 = input_dim[1:]
    reshape_x = np.reshape(data_x, (shape_1, np.prod(shape_2)))
    reshape_y = np.reshape(data_y, (shape_1, np.prod(shape_2)))
    return mse(reshape_x, reshape_y)

In [10]:
mse_batch(data_list[0], ae_pred[0], input_dim)

0.004383906

In [11]:
lol = [mse_batch(data_list[x], ae_pred[x], input_dim) for x in range(len(ae_pred))]

In [12]:
lol

[0.004383906,
 0.0041207634,
 0.002393963,
 0.0032601724,
 0.0036209936,
 0.0032147497,
 0.002727217,
 0.0066739847,
 0.0042357794,
 0.004513352,
 0.0018895474,
 0.0028035424,
 0.0017293107,
 0.004119024,
 0.0011221825,
 0.0014930187]

In [13]:
np.mean(lol)

0.0032688442

In [14]:
data = data_in.next()
data_list = data[0]
data_size = len(data_list)

/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/aogorodnikov/anaconda3/envs/imgSudoku/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


In [15]:
autoencoder.evaluate_generator(data_in)[0]

0.003834370367603595

## Notes  
We create a function that takes a single class (cellular phenotype) and test reconstruction loss for autoencoder and discriminator

In [7]:
def anomaly_score(cell_pheno draw = true):
    [shutil.rmtree(x) for x in glob("/home/aogorodnikov/test/*")]
    input_dir = os.path.join("/home/aogorodnikov/classes_rgb_corrected/", cell_pheno)
    shutil.copytree(input_dir, os.path.join("/home/aogorodnikov/test/", cell_pheno))
    img_wd = "/home/aogorodnikov/test"
    batch = 56
    input_dim = (104,104,3)
    data_loader = ImageDataGenerator(
        rescale=1. / 255,
        featurewise_center=True,
        featurewise_std_normalization=True,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

    train_data = data_loader.flow_from_directory(
        img_wd,
        target_size=(input_dim[0], input_dim[0]),
        batch_size=batch,
        class_mode='input')
    
    batch_index = 0
    discriminator_batch_losses = []
    while batch_index <= train_data.batch_index:
        data = train_data.next()
        data_list = data[0]
        data_size = len(data_list)

        fake_latent = encoder.predict(data_list)
        discriminator_input = np.concatenate((fake_latent, np.random.randn(data_size, latent_dim) * 5.))
        discriminator_labels = np.concatenate((np.zeros((data_size, 1)), np.ones((data_size, 1))))
        discriminator_history = discriminator.evaluate(x=discriminator_input, y=discriminator_labels)

            
        batch_index = batch_index + 1
        discriminator_batch_losses.append(discriminator_history[0])
    
    
    
    
    ae_res = autoencoder.evaluate_generator(train_data)[0]
    adv_res = np.mean(discriminator_batch_losses)
    
    if draw:
        fig = plt.figure(figsize=(5 * len(samples), 5))
        gs = gridspec.GridSpec(1, len(samples))
        for i, sample in enumerate(samples):
            ax = plt.Subplot(fig, gs[i])
            ax.imshow((sample["image"] * 255.).astype("int"), cmap='gray')
            ax.set_xticks([])
            ax.set_yticks([])
            ax.set_aspect('equal')
            ax.set_title(sample["title"])
            fig.add_subplot(ax)
        plt.show(block=False)
        
    
    print('Class: {}\nae_loss: {}\nadv_loss: {}'.format(cell_pheno, ae_res, adv_res))
    return [ae_res, adv_res]
    
        

SyntaxError: invalid syntax (<ipython-input-7-5058e3a51246>, line 1)